# Converting the PDF files into vector database

# 1. Initial setup

In [1]:
# Initial imports
from dotenv import load_dotenv
import os
import re
from IPython.display import display, Markdown
import json

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = os.getenv('ANTHROPIC_API_KEY')
os.environ["MATHPIX_API_ID"] = os.getenv('MATHPIX_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')

from common.utils import (
    pdf_to_md,
    split_mds,
)

from common.summarizer import summarize_collections

# Langchain framework
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.document_loaders import TextLoader

## Initial variable setup
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

USE_Anthropic = True

if USE_Anthropic:
    llm = ChatAnthropic(model_name="claude-3-sonnet-20240229", temperature=0)
else:
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # context window size 16k for GPT 3.5 Turbo

# 2. Load PDF files and convert to a vector DB

## 2.1a Convert PDFs to markdown files

In [2]:
############################################################################
# Run only to convert pdf to markdown files
############################################################################

collection_list=[
    #"Cayman",
    # "AI_BigData",
    # "ASOP_life",
    # "Bermuda",
    # "CFT",
    # "GAAP",
    # "RiskFinance",
    "PBR",
    # "VM21",
    # "VM22",
    # "Asset",
    # "IFRS17",
    # "SAP",
]
for collection_name in collection_list:
    # Put new files in the upload subfolder
    folder_path = './data/upload/pdf/'+collection_name
    download_path = './data/upload/md/'+collection_name
    os.makedirs(download_path, exist_ok=True)

    # Use loader option 5 to use Mathpix OCR to load formula, tables
    pdf_to_md(folder_path, download_path, loader_option = 5)

Status: loaded, waiting for processing to complete


## 2.1b Image formatter in Markdown files

In [3]:
def process_img_script(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    def replace_image_link(match):
        url = match.group(1)
        return f'<img src="{url}" alt="image" style="width:100%;height:auto;">'

    modified_content = re.sub(r"!\[\]\((.*?)\)", replace_image_link, content)

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(modified_content)


def process_img_mds(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                process_img_script(file_path)
                print(f"Processed: {file_path}")

# Specify the folder path containing the .md files
#folder_path = './data/md/test'
for collection_name in collection_list:
    folder_path = './data/upload/md/'+collection_name
    process_img_mds(folder_path)

Processed: ./data/upload/md/PBR/2024-NAIC-GOES-field-test-instructions.md
Processed: ./data/upload/md/PBR/2024-NAIC-VM22 Field Test Specs.md


## 2.1c Summarize the markdown files

In [4]:
summarize_collections(collection_list)

## 2.2 Set up pincone

In [5]:
pc = Pinecone(api_key=pinecone_api_key)
index_name = "valact-rag"
index = pc.Index(index_name)
index.describe_index_stats()
vectorstore = PineconeVectorStore(
    index = index,
    embedding=embeddings_model,
)
type(vectorstore)

langchain_pinecone.vectorstores.PineconeVectorStore

### Tests

In [ ]:
query = "Explain contractual service margin"  
vectorstore.similarity_search(  
    query,  # our search query  
    k=1,  # return 3 most relevant docs 
    namespace="IFRS17", 
) 

In [ ]:
from langchain.chains import RetrievalQA
query = "Explain contractual service margin"  

vectorstore = PineconeVectorStore(
    index = index,
    embedding=embeddings_model,
    namespace="IFRS17",
)

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  
qa.invoke(query) 

## 2.3a Loading markdown files to vector database (use Header Text Splitter)

In [6]:
############################################################################
# Run to load markdown files to vector database
############################################################################
# collection_list=[
#     # "Cayman",
#     # "AI_BigData",
#     # "ASOP_life",
#     "Bermuda",
#     # "CFT",
#     # "GAAP", # use this
#     # "RiskFinance",
#     # "PBR",
#     # "Asset",
#     # "IFRS17",
#     # "Product",
#     # "SAP"
# ]

for collection_name in collection_list:
    folder_path = './data/upload/md/'+collection_name

    # Call the function to load and extract text from PDFs in the specified folder
    splits = split_mds(
        folder_path, 
        IsSemantic=True,
        breakpoint_threshold_type_input="standard_deviation",
        breakpoint_threshold_amount_input=2,
        embeddings_model=embeddings_model,
    )
    # BREAKPOINT_DEFAULTS: Dict[BreakpointThresholdType, float] = {
    #     "percentile": 95,
    #     "standard_deviation": 3,
    #     "interquartile": 1.5,
    # }

    # Create a vector database from the document splits
    vectorstore.add_documents(
        documents=splits,
        namespace=collection_name,
    )


## 2.3b Update document_list.json file to be used in the Streamlit Selector
You have to move the pdf files under the pdf folder

In [7]:
def save_json(data, file_path):
    with open(file_path, "w") as file:
        json.dump(data, file)

def scan_directory(base_path):
    folders_files = {}
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            files = ["All"]
            for file in os.listdir(folder_path):
                # Exclude system files like .DS_Store
                if file != ".DS_Store":
                    files.append(file)
            files[1:] = sorted(files[1:])
            folders_files[folder] = files
    return folders_files
base_path = "./data/pdf"

document_list = scan_directory(base_path)

save_json(document_list, "./data/document_list.json")


## 2.3c json file consistency check

In [8]:
import json

# Read the link.json file
with open("./data/document_link.json") as link_file:
    link_data = json.load(link_file)

# Read the list.json file
with open('./data/document_list.json') as list_file:
    list_data = json.load(list_file)

# Loop through each collection in the list.json file
for collection, items in list_data.items():
    print(f"Checking collection: {collection}")
    
    # Loop through each item in the collection
    for item in items:
        # Check if the item exists as a key in the link.json file
        if item not in link_data:
            print(f"  Item not found: {item}")

Checking collection: SAP
Checking collection: ASOP_life
Checking collection: Asset
Checking collection: Bermuda
Checking collection: AI_BigData
Checking collection: CFT
Checking collection: Product
Checking collection: Cayman
Checking collection: RiskFinance
Checking collection: IFRS17
Checking collection: GAAP
Checking collection: PBR


## 2.3d Move MD files under the MD folder

## 2.4 Pinecone Status

In [8]:
index.describe_index_stats()

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'AI_BigData': {'vector_count': 780},
                'ASOP_life': {'vector_count': 904},
                'Asset': {'vector_count': 408},
                'Bermuda': {'vector_count': 1056},
                'CFT': {'vector_count': 501},
                'Cayman': {'vector_count': 190},
                'GAAP': {'vector_count': 831},
                'IFRS17': {'vector_count': 161},
                'PBR': {'vector_count': 1311},
                'Product': {'vector_count': 1463},
                'RiskFinance': {'vector_count': 1530},
                'SAP': {'vector_count': 267}},
 'total_vector_count': 9402}

## 2.5 Pinecone Namespace Deletion

In [ ]:
collection_list=[
    # "Cayman",
    # "AI_BigData",
    # "ASOP_life",
    # "Bermuda",
    # "CFT",
    # "GAAP",
    # "RiskFinance",
    # "PBR",
    # "VM21",
    # "VM22",
    # "Asset",
    # "IFRS17",
    # "Product",
]

for collection_name in collection_list:
    index.delete(delete_all=True, namespace=collection_name)

# 3. For test purposes

## 3.1. Retrieve from the vector store

In [ ]:
## Retrieve and RAG chain
# Create a retriever using the vector database as the search source
# You may choose a specific document to filter the search
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={
                                        'k': 6, 
                                        'lambda_mult': 0.5,
                                        # 'filter': {'source': '201611-Guidance-Notes-for-Commercial-Insurers-and-Groups-Statutory-Reporting-Regime-30-Nov-2016.pdf'}
                                        }
                                    ) 
# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
qa_system_prompt = """You are a helpful assistant to help actuaries with question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
ASOP or asop means Actuarial Standards of Practice. \
CFT means Cash Flow Testing. AAT means Asset Adequacy Testing. \
BMA means Bermuda Monetary Authority. \
SBA means scenario-based approach. BEL means best estimate liabilities.\
After you answer, provide the sources you used to answer the question. \
If you don't know the answer, just say that you don't know. \

{context}"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
)

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    #sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## 3.3. Generate Q&A Function

In [ ]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Answer\n{}\n\n### Context\n".format(output['question'], output['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            #markdown_output += "- **Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            markdown_output += "- **Source {}**: {}:\n\n{}\n".format(i, doc.metadata['source'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content
    
    # Display the Markdown output
    display(Markdown(markdown_output))

### Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [ ]:
generate_output()

# 4. Management of the vector database

In [ ]:
index.describe_index_stats()

In [ ]:
# Delete all documents from the namespace
index.delete(delete_all=True, namespace='IFRS17')

## 4.1a Remove documents from the vector database

In [ ]:
metadata_filter = {"source": "2020-NAIC-AG48.pdf"}
namespace_name = 'PBR'

# Query the index with the metadata filter
results = index.query(
    vector=[0.0]*3072, 
    top_k=10000, 
    include_metadata=True, 
    filter=metadata_filter,
    namespace=namespace_name,
)
ids = [result.id for result in results['matches']]
for id in ids:
    print(id)
index.delete(ids=ids, namespace=namespace_name)

## 4.1b Rename a namespace

In [ ]:
metadata_filter = {"source": "202307-NAIC-VM-22 Subgroup Draft.pdf"}
namespace_name = 'VM22'
new_namespace_name = 'PBR'

# Query the index with the metadata filter
results = index.query(
    vector=[0.0]*3072, 
    top_k=10000, 
    include_metadata=True, 
    filter=metadata_filter,
    namespace=namespace_name,
)

ids = [result.id for result in results['matches']]
fetch_data = index.fetch(ids=ids, namespace=namespace_name)
for id in ids:
    print(id)
    values = fetch_data.vectors[id].values
    metadata = fetch_data.vectors[id].metadata
    index.upsert(
        vectors = [
            {
                "id": id,
                "values": values,
                "metadata": metadata,
            }
        ],
        namespace=new_namespace_name
    )
index.delete(ids=ids, namespace=namespace_name)

## 4.1c Update part of info - rename metadata

In [ ]:
old_name = "2023-12-20-11-08-13-2023-Year-End-Long-Term-Instructions-Handbook.pdf"
new_name = "2023-BMA-2023YE-Long-Term-Instructions-Handbook.pdf"

metadata_filter = {"source": old_name}
namespace_name = 'Bermuda'

# Query the index with the metadata filter
results = index.query(
    vector=[0.0]*3072, 
    top_k=10000, 
    include_metadata=True, 
    filter=metadata_filter,
    namespace=namespace_name,
)

ids = [result.id for result in results['matches']]
fetch_data = index.fetch(ids=ids, namespace=namespace_name)
for id in ids:
    print(id)
    index.update(
        id=id, 
        set_metadata={"source": new_name}, 
        namespace=namespace_name
    )        

In [ ]:
#index.fetch(ids=[ids[0]], namespace=namespace_name)

In [ ]:
# json file summary update
with open('summary.json', 'r') as f:
    data = json.load(f)
# Update the existing data with the new key-value pair
data[new_name] = data.pop(old_name)

with open('summary.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:

# md and pdf file update
# Construct the paths to the folders
md_folder = os.path.join("data/md", namespace_name)
pdf_folder = os.path.join("data/pdf", namespace_name)

# Construct the old and new file paths
old_file_name_md = old_name.replace(".pdf", ".md")
new_file_name_md = new_name.replace(".pdf", ".md")
old_md_path = os.path.join(md_folder, old_file_name_md)
new_md_path = os.path.join(md_folder, new_file_name_md)

# Check if the old MD file exists before renaming
if os.path.exists(old_md_path):
    # Rename the MD file
    os.rename(old_md_path, new_md_path)
else:
    print(f"Skipping MD file rename: {old_md_path} does not exist.")

# Construct the old and new file paths
old_pdf_path = os.path.join(pdf_folder, old_name)
new_pdf_path = os.path.join(pdf_folder, new_name)

# Rename the file
os.rename(old_pdf_path, new_pdf_path)

## 4.2 Print IDs for a document

In [ ]:
metadata_filter = {"source": "asop014_082_whenToCFT.pdf"}
namespace_name = 'IFRS17'

# Query the index with the metadata filter
results = index.query(
    vector=[0.0]*3072, 
    top_k=10000, 
    include_metadata=True, 
    filter=metadata_filter,
    namespace=namespace_name,
)

ids = [result.id for result in results['matches']]
for id in ids:
    print(id)

## 4.3 Get file names from a namespace

In [37]:
namespace_name = 'RiskFinance'
response = index.query(
    vector=[0.0]*3072,
    top_k=10000,
    include_metadata=True,
    namespace=namespace_name,
)

# Extract the distinct "source" names from the response
sources = [result["metadata"]["source"] for result in response["matches"]]

distinct_sources = list(set(sources))
print("Distinct sources:", distinct_sources)

Distinct sources: ['2016-SOA-Economic-Capital-report.pdf', '2018-AAA-Life_Reins_Reserve_Credit_Practice_Note.pdf', '2022-SOA-RW_vs_RN_scenarios.pdf', '2016-AAA-ORSA_Risk_Exposures_PracticeNote.pdf', '2023-SOA-rating-agency-perspectives.pdf', '2020-AAA-Tax_Cuts_Jobs_ACT_White_Paper.pdf', '2011-SOA-Model_efficiency_study.pdf', '2023-SOA-SP_Global-Capital-Model-Change.pdf', '2024-AAA-liquidity-risk_practice_note.pdf', '2016-SOA-ESG_guide.pdf', '2009-SOA-EmbeddedValue_practice_theory.pdf', '2016-SOA-nested-stochastic-modeling-report.pdf', '2023-AAA-issue-brief-dividend-limitation.pdf', '2024-AAA-issue-brief-group-capital.pdf', '2014-SOACIA-ModelValidation_RIsk_Capital_Models.pdf', '2019-AAA-ModelRiskManagementPracticeNote.pdf', '2010-SOA-OperationalRiskPaper.pdf', '2019-IAIS-ICS-V2-Level-1-Doc.pdf', '2023-SOA-interest-rate-model-calibration.pdf', '2023-SOA-reg-capital-comparison.pdf']


# 5. Tests - to remove

In [ ]:
import matplotlib.pyplot as plt

# Create a list to store the text lengths for each split
text_lengths = []
split_lentgh = len(splits)
print(split_lentgh)
for i, doc in enumerate(splits):
    text_length = len(doc.page_content)
    text_lengths.append(text_length)
    if(text_length>40960):
        print(doc.metadata['source'])

# Create a histogram of text lengths
plt.figure(figsize=(10, 6))
plt.hist(text_lengths, bins=20, edgecolor='black')
plt.xlabel('Text Length (bytes)')
plt.ylabel('Frequency')
plt.title('Histogram of Text Lengths')
plt.grid(True)
plt.show()

In [ ]:
splits[11]

In [ ]:
md_file = "./data/md/GAAP/2005-AAA-DIGB36_PracticNote.md"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False,
)
loader = TextLoader(md_file)
loaded_docs = loader.load()
# MD splits
md_header_splits = markdown_splitter.split_text(loaded_docs[0].page_content)
len(md_header_splits)
